Užduotį atliko: Lukas Pupelis 2110612.

Klasių rinkinys: Zebras (zebra), Medūza (jellyfish), Žąsis(goose).

Nuotraukų įkėlimui naudojame OpenImages biblioteką.

In [ ]:
!pip install openimages


Importuojam bibliotekas

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import numpy as np
import PIL

Tikrinam, ar veikia GPU, jei ne, imam CPU.

In [13]:
torch.cuda.is_available()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

Nustatome aplanko pavadinimą, nuotraukų kiekį klasėje ir klasių pavadinimus. Klasių ID imti iš https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/

In [ ]:
data_dir = "data"
number_for_samples = 600
classes = ["Goose", "Zebra", "Jellyfish"]
class_ids = [99, 340, 107]

Sukuriamas aplankas nuotraukoms.

In [ ]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

Jos atsisiunčiamos.

In [ ]:
print("Downloading is starting...")
download_dataset(data_dir, classes, limit=number_for_samples)

Transformacijos

In [ ]:
# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize(256, interpolation=PIL.Image.BILINEAR),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Duomenų pakrovimas ir duomenų aibės skėlimas į dvi dalis(testavimo ir vertinimo)

In [ ]:
# Load the dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into train and test subsets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.5, random_state=42)

# Create data loaders for train and test subsets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Modelio apibrėžimas

In [ ]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, len(classes))

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Modelio inicializavimas ir jo treniravimas

In [ ]:
# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss}")

Testavimas su abiejais duomenų rinkiniais.

In [ ]:
 # Evaluation on train dataset
    model.eval()
    train_predictions = []
    train_targets = []
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_predictions.extend(predicted.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())
    train_accuracy = accuracy_score(train_targets, train_predictions)
    train_precision = precision_score(train_targets, train_predictions, average='macro')
    train_recall = recall_score(train_targets, train_predictions, average='macro')
    train_f1_score = f1_score(train_targets, train_predictions, average='macro')
    train_confusion_matrix = confusion_matrix(train_targets, train_predictions)

    print("Training Metrics:")
    print("Accuracy:", train_accuracy)
    print("Precision:", train_precision)
    print("Recall:", train_recall)
    print("F1 Score:", train_f1_score)
    print("Confusion Matrix:")
    print(train_confusion_matrix)

    # Evaluation on test dataset
    test_predictions = []
    test_targets = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.cpu().numpy())
            test_targets.extend(labels.cpu().numpy())
    test_accuracy = accuracy_score(test_targets, test_predictions)
    test_precision = precision_score(test_targets, test_predictions, average='macro')
    test_recall = recall_score(test_targets, test_predictions, average='macro')
    test_f1_score = f1_score(test_targets, test_predictions, average='macro')
    test_confusion_matrix = confusion_matrix(test_targets, test_predictions)

    print("Testing Metrics:")
    print("Accuracy:", test_accuracy)
    print("Precision:", test_precision)
    print("Recall:", test_recall)
    print("F1 Score:", test_f1_score)
    print("Confusion Matrix:")
    print(test_confusion_matrix)